# Linear Regression Tutorial

Author: Andrew Andrade ([andrew@andrewandrade.ca](mailto:andrew@andrewandrade.ca))

This is part one of a series of tutorials related to regression used in data science.

In this tutorial we will fit a simple line using Least Squares Linear Regression (LSLR), examine residuals, plot distributions, compare vertical vs horizontal vs perpendicular residuals, and end with total least squares.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import log

# Try to import sklearn, else install
try:
    from sklearn import linear_model
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install scikit-learn
    from sklearn import linear_model

%matplotlib inline

Read the data and display it.

In [2]:
anscombe_i = pd.read_csv('anscombe_i.csv')
anscombe_i

In [3]:
plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.xlabel('X')
plt.ylabel('Y')

## Fit Linear Regression using sklearn


In [4]:
regr_i = linear_model.LinearRegression()

X = anscombe_i.x.values.reshape(-1, 1)
y = anscombe_i.y.values.reshape(-1, 1)

regr_i.fit(X, y)

print('Coefficient (m):', regr_i.coef_[0][0])
print('Intercept (b):', regr_i.intercept_[0])
print('Residual Sum of Squares:', np.mean((regr_i.predict(X) - y) ** 2))
print('Variance Score:', regr_i.score(X, y))

plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.plot(X, regr_i.predict(X), color='green', linewidth=3)
plt.xlabel('X')
plt.ylabel('Y')

## Plot Residuals (Vertical)

In [5]:
from numpy import polyfit

k, d = polyfit(anscombe_i.x, anscombe_i.y, 1)
yfit = k * anscombe_i.x + d

plt.figure(figsize=(7,5))
plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.plot(anscombe_i.x, yfit, color='green')

for i in range(len(anscombe_i)):
    plt.plot([anscombe_i.x[i], anscombe_i.x[i]], [anscombe_i.y[i], yfit[i]], 'k')

plt.xlabel('X')
plt.ylabel('Y')

## Residual Distribution

In [6]:
residual = anscombe_i.y - yfit
mean_r = np.mean(residual)
std_r = np.std(residual)

plt.figure(figsize=(7,5))
plt.scatter(anscombe_i.x, residual)
plt.xlabel('X')
plt.ylabel('Residual')

plt.figure(figsize=(7,5))
plt.hist(residual, bins=10, density=True, alpha=0.75)

from scipy.stats import norm
xvals = np.linspace(min(residual), max(residual), 100)
plt.plot(xvals, norm.pdf(xvals, mean_r, std_r), 'k--')
plt.xlabel('Residual Error')
plt.title('Residual Distribution')

## Statsmodels OLS

In [7]:
import statsmodels.api as sm

X = sm.add_constant(anscombe_i.x)
model = sm.OLS(anscombe_i.y, X).fit()
model.summary()

## Statsmodels Regression Line

In [8]:
plt.scatter(anscombe_i.x, anscombe_i.y, color='black')

xp = np.linspace(anscombe_i.x.min(), anscombe_i.x.max(), 100)
Xp = sm.add_constant(xp)
plt.plot(xp, model.predict(Xp), 'r')

## Seaborn Regression + Marginals

In [9]:
import seaborn as sns
sns.set(style='darkgrid', color_codes=True)

sns.jointplot(data=anscombe_i, x='x', y='y', kind='reg', height=6, color='red')

## Horizontal Residual Regression

In [10]:
k2, d2 = polyfit(anscombe_i.y, anscombe_i.x, 1)
xfit = k2 * anscombe_i.y + d2

plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.plot(xfit, anscombe_i.y, 'blue')

for i in range(len(anscombe_i)):
    plt.plot([anscombe_i.x[i], xfit[i]], [anscombe_i.y[i], anscombe_i.y[i]], 'k')

plt.xlabel('X')
plt.ylabel('Y')

## Total Least Squares (Orthogonal Regression)

In [11]:
from scipy.odr import Model, Data, ODR
from scipy.stats import linregress

def fit_function(p, x):
    return p[0] * x + p[1]

def orthoregress(x, y):
    lr = linregress(x, y)
    model = Model(fit_function)
    data = Data(x, y)
    od = ODR(data, model, beta0=lr[0:2])
    out = od.run()
    return out.beta

m, b = orthoregress(anscombe_i.x.values, anscombe_i.y.values)

y_ortho = m * anscombe_i.x + b

plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.plot(anscombe_i.x, y_ortho, 'r')
plt.xlabel('X')
plt.ylabel('Y')

## Compare All Three Regression Lines

In [12]:
plt.scatter(anscombe_i.x, anscombe_i.y, color='black')
plt.plot(anscombe_i.x, yfit, 'g', label='Vertical Residuals')
plt.plot(xfit, anscombe_i.y, 'b', label='Horizontal Residuals')
plt.plot(anscombe_i.x, y_ortho, 'r', label='Total Least Squares')

plt.legend()
plt.xlabel('X')
plt.ylabel('Y')